In [3]:
!pip install pytorch-lightning 

     |████████████████████████████████| 582 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 72.8 MB/s 
     |████████████████████████████████| 136 kB 82.1 MB/s 
     |████████████████████████████████| 408 kB 90.6 MB/s 
     |████████████████████████████████| 1.1 MB 75.1 MB/s 
     |████████████████████████████████| 144 kB 89.9 MB/s 
     |████████████████████████████████| 271 kB 77.8 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import random
import torchvision.transforms.functional as TF


import pytorch_lightning as pl
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics.functional import accuracy
from torch.optim import Adam
from torch.utils.data import Sampler

import numpy as np
from sklearn.metrics import classification_report
from collections import OrderedDict
from typing import Sized, Iterator

In [12]:
class Lenet5(nn.Module):
    
    
    def __init__(self)->None:
        super(Lenet5, self).__init__()
        
        self.layer1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, 6, kernel_size=(5, 5))),
            ('relu1', nn.ReLU()),
            ('pool1', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.layer2 = nn.Sequential(OrderedDict([
            ('conv2', nn.Conv2d(6, 16, kernel_size=(5, 5))),
            ('relu2', nn.ReLU()),
            ('pool2', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.fc1 = nn.Sequential(OrderedDict([
            ('f4', nn.Linear(400, 84)),
            ('relu4', nn.ReLU())
        ]))
        
        self.fc2 = nn.Sequential(OrderedDict([
            ('f5', nn.Linear(84, 10)),
        ]))
        
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [13]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [14]:
surrogate = Lenet5()
surrogate.to(device)

Lenet5(
  (layer1): Sequential(
    (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (f4): Linear(in_features=400, out_features=84, bias=True)
    (relu4): ReLU()
  )
  (fc2): Sequential(
    (f5): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [163]:
class BatchReorderSampler(Sampler[int]):

    def __init__(self, data_source: Sized, atk_type="lowtohigh", surrogate= surrogate, batch_size=32) -> None:
        self.data_source = data_source
        
        self.surrogate = surrogate

        self.epoch1 = True
        
        self.batchOrder = torch.randperm((len(data_source)//batch_size)*batch_size)
        self.batchOrder = self.batchOrder.reshape(-1, batch_size)
        
        data = [self.data_source.__getitem__(j) for j in self.batchOrder.view(-1)]
        data, labels = zip(*data)
        self.data = torch.stack(data).to(device)
        self.labels = torch.LongTensor(labels).to(device)
        self.atk_type = atk_type
     
    def __getSurrogateloss__(self, batch):
        
        
        with torch.no_grad():
            
            loss = F.nll_loss(self.surrogate(self.data[batch]) ,self.labels[batch])
        
        return loss.cpu().item()
    
    
    def __iter__(self) -> Iterator[int]:
        
        
        if self.epoch1 == True:
            print('Waiting to Attack')
            for i in range(self.batchOrder.shape[0]):
                yield iter(self.batchOrder[i])
            
            self.epoch1 = False
            
        else:
            print('Attacking')
            print("batch order: ", self.batchOrder)
            losses = torch.Tensor([self.__getSurrogateloss__(batch) for batch in self.batchOrder])
            if self.atk_type == "lowtohigh":
              for i in torch.argsort(losses):
                yield iter(self.batchOrder[i])
            elif self.atk_type == "hightolow":

              for i in torch.argsort(losses, descending=True):
                  yield iter(self.batchOrder[i])
            elif self.atk_type == "oscillating in":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_inward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_inward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_inward_sort.append(asc_sort_losses[loss_ind])
              oscillate_inward_sort_tensor = torch.tensor(np.array(oscillate_inward_sort))
              for i in oscillate_inward_sort_tensor:
                   yield iter(self.batchOrder[i])
            elif self.atk_type == "oscillating out":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_outward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_outward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_outward_sort.append(asc_sort_losses[loss_ind])
              oscillate_outward_sort_tensor = torch.tensor(np.array(oscillate_outward_sort))
              oscillate_outward_sort_tensor = torch.flip(oscillate_outward_sort_tensor, [0])
              for i in oscillate_outward_sort_tensor:
                  yield iter(self.batchOrder[i])
            
        

    def __len__(self) -> int:
        return self.batchOrder.shape[0]

In [118]:
temp_torch_arr = torch.tensor(np.array([1, 2, 3,4, 5, 6]))
# temp_torch_arr = temp_torch_arr.view(-1, 2)
torch.flip(temp_torch_arr, [0])

tensor([6, 5, 4, 3, 2, 1])

In [158]:
class BatchShuffleSampler(Sampler[int]):

    def __init__(self, data_source: Sized,  atk_type="lowtohigh", surrogate=surrogate, batch_size=64) -> None:
        self.data_source = data_source
        
        self.surrogate = surrogate
        self.batch_size = batch_size

        self.epoch1 = True
        self.batchOrder = torch.randperm((len(data_source)//batch_size)*batch_size)
        
        data = [self.data_source.__getitem__(j) for j in self.batchOrder.view(-1)]
        data, labels = zip(*data)
        self.data = torch.stack(data).to(device)
        self.labels = torch.LongTensor(labels).to(device)

        self.atk_type = atk_type
     
    def __getSurrogateloss__(self, batch):
        
        
        with torch.no_grad():
            # random_batch_perm = torch.randperm(batch_size)
            # data_perm = self.data[random_batch_perm]
            # label_perm = self.labels[random_batch_perm]
            
            loss = F.nll_loss(self.surrogate(self.data[batch: batch+1]) 
            ,self.labels[batch: batch+1])
        
        return loss.cpu().item()
    
    
    def __iter__(self) -> Iterator[int]:
        
        
        if self.epoch1 == True:
            print('Waiting to Attack')
            
            for i in self.batchOrder.view(-1, self.batch_size):
                yield iter(i)
            
            self.epoch1 = False
            
        else:
            
            print('Attacking')
            # print("batch order: ",self.batchOrder)
            losses = torch.Tensor([self.__getSurrogateloss__(batch) for batch in self.batchOrder])
            # print(losses)
            if self.atk_type == "lowtohigh":
              for i in self.batchOrder[torch.argsort(losses)].view(-1, self.batch_size):
                  yield iter(i)
            elif self.atk_type == "hightolow":
              print(self.batchOrder.shape)
              print(self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size).shape)
              for i in self.batchOrder[torch.argsort(losses, descending=True)].view(-1, self.batch_size):
                  yield iter(i)
            elif self.atk_type == "oscillating in":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_inward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_inward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_inward_sort.append(asc_sort_losses[loss_ind])
              oscillate_inward_sort_tensor = torch.tensor(np.array(oscillate_inward_sort))
              for i in self.batchOrder[oscillate_inward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            elif self.atk_type == "oscillating out":
              asc_sort_losses = torch.argsort(losses).tolist()
              oscillate_outward_sort = []
              asc_n = len(asc_sort_losses) - 1
              for loss_ind in range(asc_n//2 + 1):
                oscillate_outward_sort.append(asc_sort_losses[asc_n - loss_ind])
                oscillate_outward_sort.append(asc_sort_losses[loss_ind])
              oscillate_outward_sort_tensor = torch.tensor(np.array(oscillate_outward_sort))
              oscillate_outward_sort_tensor = torch.flip(oscillate_outward_sort_tensor, [0])
              for i in self.batchOrder[oscillate_outward_sort_tensor].view(-1, self.batch_size):
                  yield iter(i)
            print('Attack successful')
        

    def __len__(self) -> int:
        return self.batchOrder.view(-1, self.batch_size).shape[0]

In [176]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_transform = transforms.Compose(
    [
     torchvision.transforms.RandomCrop(32, padding=4),
#      torchvision.transforms.Resize(),
     torchvision.transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

attack_type = ["lowtohigh", "hightolow", "oscillating in", "oscillating out"]

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchReorderSampler(trainset, atk_type=attack_type[3]))
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                          num_workers=16, batch_sampler=BatchReorderSampler(trainset))#, shuffle=True

# trainloader = torch.utils.data.DataLoader(trainset, 
#                                           num_workers=16, batch_sampler=BatchShuffleSampler(trainset, attack_type[3]))


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [177]:
def resnet18CIFAR10():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    
    return model

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()

        self.train_l = train_l
        self.val_l = val_l
        
        self.model = resnet18CIFAR10()
        
        
        self.surrogate = surrogate
        self.surrogate_optim = Adam([p for p in self.surrogate.parameters() if p.requires_grad],lr=0.1)
        
        

    def forward(self, x):
        out = self.model(x)
        
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        
        
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = F.nll_loss(surrogate_logits, y)
        surrogateloss.backward()
        self.surrogate_optim.step()
        
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")


    def configure_optimizers(self):
        return Adam([p for p in self.model.parameters() if p.requires_grad], lr=0.1)


    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

In [178]:
model = BoilerPlate(trainloader, testloader, surrogate)
trainer = pl.Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=2,
    gpus=1,
    logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type   | Params
-------------------------------------
0 | model     | ResNet | 11.2 M
1 | surrogate | Lenet5 | 37.4 K
-------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

Waiting to Attack


Validation: 0it [00:00, ?it/s]

Attacking
batch order:  tensor([[18877, 17294, 28343,  ..., 42071, 31506, 30023],
        [15953, 40502, 27451,  ..., 31728, 17793, 47608],
        [49724, 34006, 22283,  ..., 42381, 24636,  2062],
        ...,
        [11521, 26677, 46991,  ..., 12354, 34361, 32453],
        [ 2675, 26992, 41969,  ..., 26190, 42211, 14095],
        [27083, 25715,  8248,  ..., 28470,  6307, 23085]])


Validation: 0it [00:00, ?it/s]

In [179]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


batch_size = 64
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


              precision    recall  f1-score   support

       plane       0.32      0.35      0.34      1000
         car       1.00      0.01      0.02      1000
        bird       0.23      0.58      0.32      1000
         cat       0.00      0.00      0.00      1000
        deer       0.12      0.14      0.13      1000
         dog       0.24      0.32      0.28      1000
        frog       0.00      0.00      0.00      1000
       horse       0.34      0.12      0.17      1000
        ship       0.38      0.77      0.51      1000
       truck       0.40      0.56      0.46      1000

    accuracy                           0.29     10000
   macro avg       0.30      0.29      0.22     10000
weighted avg       0.30      0.29      0.22     10000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
